Import Libraries

In [2]:
import pandas as pd
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler
from sklearn.preprocessing import LabelEncoder
from sklearn.metrics import classification_report
from keras.models import Sequential
from keras.layers import Dense
from sklearn.model_selection import GridSearchCV

Data Analysis

In [3]:
data = pd.read_csv('mobile_price_classification.csv')

In [4]:
data.head()

,battery_power,bluetooth,clock_speed,dual_sim,front_cam,4G,int_memory,m_dep,mobile_wt,n_cores,...,px_height,px_width,ram,sc_h,sc_w,talk_time,three_g,touch_screen,wifi,price_range
0,842,0,2.2,0,1,0,7,0.6,188,2,...,20,756,2549,9,7,19,0,0,1,1
1,1021,1,0.5,1,0,1,53,0.7,136,3,...,905,1988,2631,17,3,7,1,1,0,2
2,563,1,0.5,1,2,1,41,0.9,145,5,...,1263,1716,2603,11,2,9,1,1,0,2
3,615,1,2.5,0,0,0,10,0.8,131,6,...,1216,1786,2769,16,8,11,1,0,0,2
4,1821,1,1.2,0,13,1,44,0.6,141,2,...,1208,1212,1411,8,2,15,1,1,0,1


In [5]:
data.shape

(2000, 21)

In [6]:
data['price_range'].value_counts()

1    500
2    500
3    500
0    500
Name: price_range, dtype: int64

In [7]:
data.dtypes

battery_power       int64
bluetooth           int64
clock_speed       float64
dual_sim            int64
front_cam           int64
4G                  int64
int_memory          int64
m_dep             float64
mobile_wt           int64
n_cores             int64
primary_camera      int64
px_height           int64
px_width            int64
ram                 int64
sc_h                int64
sc_w                int64
talk_time           int64
three_g             int64
touch_screen        int64
wifi                int64
price_range         int64
dtype: object

In [8]:
data['price_range'].nunique()

4

Preprocess the data by scaling features and encoding labels

In [9]:
X = data.drop('price_range', axis=1)
y = data['price_range']

In [10]:
scaler = StandardScaler()
X_scaled = scaler.fit_transform(X)

In [11]:
encoder = LabelEncoder()
y_encoded = encoder.fit_transform(y)

Split the data into training and testing sets

In [12]:
X_train, X_test, y_train, y_test = train_test_split(X_scaled, y_encoded, test_size=0.2, random_state=42)

creating the model

In [13]:
def create_model(optimizer='adam', activation='relu', neurons=64):
    model = Sequential()
    model.add(Dense(neurons, input_dim=X_train.shape[1], activation=activation))
    model.add(Dense(neurons, activation=activation))
    model.add(Dense(4, activation='softmax'))
    model.compile(loss='sparse_categorical_crossentropy', optimizer=optimizer, metrics=['accuracy'])
    return model

Create KerasClassifier

In [15]:
!pip install scikeras
from scikeras.wrappers import KerasClassifier

In [16]:
model = KerasClassifier(build_fn=create_model, verbose=0)

Grid Search

In [21]:
optimizer = ['SGD','Adam','Adamax','Nadam']
epochs = [10,20,30]
learn_rate = [0.001,0.01,0.1,0.2,0.3]

param_grid = dict(model__optimizer = optimizer, optimizer__learning_rate=learn_rate,epochs=epochs)
param_grid

{'model__optimizer': ['SGD', 'Adam', 'Adamax', 'Nadam'],
 'optimizer__learning_rate': [0.001, 0.01, 0.1, 0.2, 0.3],
 'epochs': [10, 20, 30]}

In [22]:
from sklearn.model_selection import GridSearchCV
grid = GridSearchCV(estimator=model, param_grid=param_grid, n_jobs=-1, cv=3)
grid_result = grid.fit(X_train, y_train)

/usr/local/lib/python3.10/dist-packages/joblib/externals/loky/process_executor.py:752: UserWarning: A worker stopped while some jobs were given to the executor. This can be caused by a too short worker timeout or by a memory leak.
  warnings.warn(
/usr/local/lib/python3.10/dist-packages/scikeras/wrappers.py:915: UserWarning: ``build_fn`` will be renamed to ``model`` in a future release, at which point use of ``build_fn`` will raise an Error instead.
  X, y = self._initialize(X, y)


In [23]:
print("Best: %f using %s" % (grid_result.best_score_, grid_result.best_params_))
means = grid_result.cv_results_['mean_test_score']
stds = grid_result.cv_results_['std_test_score']
params = grid_result.cv_results_['params']
for mean, stdev, param in zip(means, stds, params):
    print("%f (%f) with: %r" % (mean, stdev, param))

Best: 0.912498 using {'epochs': 30, 'model__optimizer': 'Nadam', 'optimizer__learning_rate': 0.001}
0.603724 (0.029951) with: {'epochs': 10, 'model__optimizer': 'SGD', 'optimizer__learning_rate': 0.001}
0.556866 (0.020978) with: {'epochs': 10, 'model__optimizer': 'SGD', 'optimizer__learning_rate': 0.01}
0.598103 (0.024754) with: {'epochs': 10, 'model__optimizer': 'SGD', 'optimizer__learning_rate': 0.1}
0.591856 (0.028507) with: {'epochs': 10, 'model__optimizer': 'SGD', 'optimizer__learning_rate': 0.2}
0.567518 (0.037845) with: {'epochs': 10, 'model__optimizer': 'SGD', 'optimizer__learning_rate': 0.3}
0.866260 (0.012912) with: {'epochs': 10, 'model__optimizer': 'Adam', 'optimizer__learning_rate': 0.001}
0.873128 (0.006134) with: {'epochs': 10, 'model__optimizer': 'Adam', 'optimizer__learning_rate': 0.01}
0.874999 (0.000994) with: {'epochs': 10, 'model__optimizer': 'Adam', 'optimizer__learning_rate': 0.1}
0.896883 (0.010634) with: {'epochs': 10, 'model__optimizer': 'Adam', 'optimizer__le

Evaluate the model on the test set

In [24]:
y_pred = grid.predict(X_test)
print(classification_report(y_test, y_pred))

              precision    recall  f1-score   support

           0       0.95      0.95      0.95       105
           1       0.90      0.91      0.91        91
           2       0.91      0.89      0.90        92
           3       0.95      0.96      0.95       112

    accuracy                           0.93       400
   macro avg       0.93      0.93      0.93       400
weighted avg       0.93      0.93      0.93       400

